#### Prepare

In [ ]:
# Create metawrap virtual env
conda create -n metawrap python=2.7
# Activate metawrap virtual env
source activate metawrap
# Set channels
conda config --add channels defaults
conda config --add channels conda-forge
conda config --add channels bioconda
conda config --add channels ursky
conda install -c ursky metawrap-mg

In [ ]:
# Set database
# CheckM db
mkdir checkm
cd checkm
wget https://data.ace.uq.edu.au/public/CheckM_databases/checkm_data_2015_01_16.tar.gz
tar -xvf *.tar.gz
rm *.gz

# NCBI_nt v4 db
mkdir NCBI_nt && cd NCBI_nt
wget -c "ftp://ftp.ncbi.nlm.nih.gov/blast/db/v4/nt*.tar.gz"
for a in nt*.tar.gz; do tar xzf $a; done

# NCBI_tax db
mkdir NCBI_tax && cd NCBI_tax
wget ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz
tar -xvf taxdump.tar.gz

# Set paths of db in the config file
which config-metawrap

In [ ]:
# Create working directory
mkdir metawrap-work
cd metawrap-work

#### Assembly

In [ ]:
# prepare reads
cp /d5/lin/Bee_microbiome/Fq/AB1.nobee_qc_1.fastq ./
cp /d5/lin/Bee_microbiome/Fq/AB1.nobee_qc_2.fastq ./

In [ ]:
# assembly (use metahit)
metawrap assembly -1 AB1.nobee_qc_1.fastq -2 AB1.nobee_qc_2.fastq --megahit -m 20 -t 20  -o AB1_ASSEMBLY
# The contigs are saved in AB1_ASSEMBLY/final_assembly.fasta.

#### Binning

In [ ]:
# binning using metabat2, maxbin2 and concoct
metawrap binning -o INITIAL_BINNING -t 8 -a AB1_ASSEMBLY/final_assembly.fasta --metabat2 --maxbin2 --concoct AB1.nobee_qc_1.fastq AB1.nobee_qc_2.fastq

#### Bin refinement

In [ ]:
# bin refinement
metawrap bin_refinement -o BIN_REFINEMENT -t 8 -A INITIAL_BINNING/metabat2_bins/ -B INITIAL_BINNING/maxbin2_bins/ -C INITIAL_BINNING/concoct_bins/ -c 50 -x 10

#### Quantify bins

In [ ]:
# Quantify bins using salmon
metawrap quant_bins -b BIN_REFINEMENT/metawrap_50_10_bins -t 8 -o QUANT_BINS -a AB1_ASSEMBLY/final_assembly.fasta AB1.nobee_qc_1.fastq AB1.nobee_qc_2.fastq

#### Reassembly

In [ ]:
# reassembly
metawrap reassemble_bins -o BIN_REASSEMBLY -1 AB1.nobee_qc_1.fastq -2 AB1.nobee_qc_2.fastq -t 8 -m 800 -c 50 -x 10 -b BIN_REFINEMENT/metawrap_50_10_bins

#### Taxonomy profile

In [ ]:
# bin taxonomy
metawrap classify_bins -b BIN_REASSEMBLY/reassembled_bins -o BIN_CLASSIFICATION -t 8

#### Functional annotation

In [ ]:
# bin function
metaWRAP annotate_bins -o FUNCT_ANNOT -t 8 -b BIN_REASSEMBLY/reassembled_bins/